In [7]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [8]:
# %cd MyDrive/FYP_CQF1/degradation/degradation
%cd MyDrive/Colab\ Notebooks/SwinGAN/stylegan2/degradation

/gdrive/MyDrive/Colab Notebooks/SwinGAN/stylegan2/degradation


In [9]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [10]:
%pip install -q omegaconf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [11]:
from utils.common import instantiate_from_config, load_state_dict
from omegaconf import OmegaConf

In [12]:
config = OmegaConf.load("./configs/general_deg_realesrgan_train.yaml")
config

{'batch_transform': {'target': 'degradation.batch_transform.RealESRGANBatchTransform', 'params': {'out_size': 256, 'crop_type': 'center', 'use_hflip': False, 'use_rot': False, 'blur_kernel_size': 21, 'kernel_list': ['iso', 'aniso', 'generalized_iso', 'generalized_aniso', 'plateau_iso', 'plateau_aniso'], 'kernel_prob': [0.45, 0.25, 0.12, 0.03, 0.12, 0.03], 'sinc_prob': 0.1, 'blur_sigma': [0.2, 3], 'betag_range': [0.5, 4], 'betap_range': [1, 2], 'blur_kernel_size2': 21, 'kernel_list2': ['iso', 'aniso', 'generalized_iso', 'generalized_aniso', 'plateau_iso', 'plateau_aniso'], 'kernel_prob2': [0.45, 0.25, 0.12, 0.03, 0.12, 0.03], 'sinc_prob2': 0.1, 'blur_sigma2': [0.2, 1.5], 'betag_range2': [0.5, 4], 'betap_range2': [1, 2], 'final_sinc_prob': 0.8, 'use_sharpener': False, 'resize_hq': False, 'queue_size': 256, 'resize_prob': [0.2, 0.7, 0.1], 'resize_range': [0.15, 1.5], 'gaussian_noise_prob': 0.5, 'noise_range': [1, 30], 'poisson_scale_range': [0.05, 3], 'gray_noise_prob': 0.4, 'jpeg_range':

In [13]:
transform = instantiate_from_config(config.batch_transform)
transform

ModuleNotFoundError: No module named 'degradation'

In [ ]:
from PIL import Image
import os
import random
from torchvision import transforms
import matplotlib.pyplot as plt

dataset_path = '/gdrive/MyDrive/dataset/df2k-ost/test/DIV2K_valid'
image_filenames = os.listdir(dataset_path)

# Randomly choose 10 images
random_images = random.sample(image_filenames, 10)

# Convert the images to PyTorch tensors
transform_toTensor = transforms.ToTensor()

# Create a subplot grid for displaying the images
fig, axs = plt.subplots(10, 2, figsize=(12, 24))
# fig, axs = plt.subplots(2, 10, figsize=(12, 24))

images = []

for i, image_filename in enumerate(random_images):
    # Open the original image using PIL
    image_path = os.path.join(dataset_path, image_filename)
    image_pil = Image.open(image_path)

    # Convert the original image to a PyTorch tensor
    image_tensor = transform_toTensor(image_pil)
    # print(type(image_tensor), image_tensor.size())
    result = transform(image_tensor.unsqueeze(0))

    lq_img = result["hint"]
    out = lq_img.to(device).float()
    out = out.detach().cpu().numpy()
    # Plot the original image on the left subplot

    axs[i, 0].imshow(image_tensor.permute(1, 2, 0))
    axs[i, 0].axis('off')

    # print(out.shape)
    # Plot the processed image on the right subplot
    axs[i, 1].imshow(out[0])
    axs[i, 1].axis('off')

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()